# 🤖 AUTOCREATE WIKI RAG システム デモ

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/USERNAME/AUTOCREATE/blob/main/AUTOCREATE_WIKI_RAG_Demo.ipynb)

## 概要

このノートブックでは、**AUTOCREATE**の既存WIKIナレッジを活用したRAG（Retrieval-Augmented Generation）システムを構築・デモンストレーションします。

### 🎯 目標
- ChromaDBを使ったベクトル検索システムの構築
- 既存WIKI文書からの自動ナレッジベース作成
- 自然言語による質問応答システムの実現

### 🚀 特徴
- **AI社長×無職CTO体制**による革新的なAI自動開発
- 多言語対応（日本語・英語）
- リアルタイム検索・回答生成
- Gradio Webインターフェース連携

---

In [ ]:
# 1️⃣ 必要なライブラリのインストール
!pip install -q chromadb>=0.4.22
!pip install -q langchain>=0.1.0
!pip install -q langchain-community>=0.0.13
!pip install -q sentence-transformers>=2.2.2
!pip install -q gradio>=4.0.0
!pip install -q python-dotenv>=1.0.0
!pip install -q markdown>=3.5.0
!pip install -q beautifulsoup4>=4.12.0

print("✅ 依存関係のインストールが完了しました！")

In [ ]:
# 2️⃣ ライブラリのインポート
import os
import sys
import logging
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any, Optional

# ChromaDB & LangChain
import chromadb
from chromadb.config import Settings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.schema import Document

# その他のライブラリ
import gradio as gr
import numpy as np
import pandas as pd
import markdown
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# 環境設定
load_dotenv()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("📚 ライブラリのインポートが完了しました！")
print(f"🗂️ ChromaDB バージョン: {chromadb.__version__}")

In [ ]:
# 3️⃣ WIKI RAGシステムクラス定義
class WikiRAGSystem:
    """AUTOCREATE WIKI RAGシステム"""
    
    def __init__(self, wiki_paths: List[str] = None, chroma_path: str = None):
        """初期化"""
        # Colab環境用のデフォルトパス設定
        if wiki_paths is None:
            wiki_paths = []
            # サンプルWIKI データがある場合
            if os.path.exists("/content/wikigit"):
                wiki_paths.append("/content/wikigit")
            if os.path.exists("/content/docs"):
                wiki_paths.append("/content/docs")
            # ローカル環境用
            if os.path.exists("/workspaces/AUTOCREATE/wikigit"):
                wiki_paths.append("/workspaces/AUTOCREATE/wikigit")
            if os.path.exists("/workspaces/AUTOCREATE/docs"):
                wiki_paths.append("/workspaces/AUTOCREATE/docs")
        
        self.wiki_paths = wiki_paths
        self.chroma_path = chroma_path or "/tmp/chroma_wiki_rag"
        self.collection_name = "wiki_knowledge"
        
        # テキスト分割器
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
            separators=["\n\n", "\n", "。", ".", " ", ""]
        )
        
        # 埋め込みモデル（多言語対応）
        self.embeddings = HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-large",
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        
        # ChromaDB設定
        self.chroma_client = None
        self.vectorstore = None
        
        print(f"🚀 WIKI RAGシステムを初期化しました")
        print(f"📁 WIKIパス: {self.wiki_paths}")
        print(f"💾 Chromaパス: {self.chroma_path}")
    
    def initialize_chroma(self):
        """ChromaDB初期化"""
        try:
            os.makedirs(self.chroma_path, exist_ok=True)
            
            self.chroma_client = chromadb.PersistentClient(
                path=self.chroma_path,
                settings=Settings(
                    anonymized_telemetry=False,
                    allow_reset=True
                )
            )
            
            # コレクション作成・取得
            try:
                collection = self.chroma_client.get_collection(name=self.collection_name)
                print(f"✅ 既存コレクション '{self.collection_name}' を取得")
            except:
                collection = self.chroma_client.create_collection(
                    name=self.collection_name,
                    metadata={"description": "AUTOCREATE WIKI Knowledge Base"}
                )
                print(f"🆕 新規コレクション '{self.collection_name}' を作成")
            
            # LangChain Vectorstore作成
            self.vectorstore = Chroma(
                client=self.chroma_client,
                collection_name=self.collection_name,
                embedding_function=self.embeddings
            )
            
            return True
            
        except Exception as e:
            print(f"❌ ChromaDB初期化エラー: {e}")
            return False

print("✅ WikiRAGSystemクラスが定義されました！")

In [ ]:
# 4️⃣ サンプルWIKIデータの作成
def create_sample_wiki_data():
    """デモ用のサンプルWIKIデータを作成"""
    
    # サンプルデータディレクトリ
    sample_dir = "/tmp/sample_wiki"
    os.makedirs(sample_dir, exist_ok=True)
    
    # サンプルWIKIコンテンツ
    wiki_contents = [
        {
            "filename": "Home.md",
            "content": """# AUTOCREATE プロジェクト

## 概要
AUTOCREATEは、世界初の「AI社長×無職CTO体制」による革新的なAI自動開発・AI視覚自動化システムです。

## 特徴
- 自然言語での開発指示が可能
- OCR + RPA による視覚自動化
- リユース業界特化の AI システム
- オープンソース戦略

## 理念
「思ったことを自然言語で伝えるだけで、AIが自動的にシステムを構築する」
これが私たちの目指す未来です。
"""
        },
        {
            "filename": "AI-Vision-Automation.md", 
            "content": """# AI視覚自動化システム

## 概要
AI視覚自動化システムは、画面認識・OCR・RPA を組み合わせた革新的な自動化ソリューションです。

## 技術スタック
- **OCR**: Google Apps Script + Vision API
- **RPA**: Python + Selenium + xdotool
- **AI分析**: LLM + Vision模型
- **UI**: Gradio + Jupyter Notebook

## ユースケース
1. **kinkaimasu.jp自動化**: 買取価格の自動取得
2. **データ入力自動化**: OCR結果の自動入力
3. **画面操作自動化**: セレクター分析による自動操作

## 導入効果
- 手作業削減: 90%以上
- 処理速度向上: 10倍以上
- エラー率削減: 95%以上
"""
        },
        {
            "filename": "Gradio-System.md",
            "content": """# Gradioシステム

## 概要
GradioはWebベースの機械学習インターフェースを簡単に作成できるPythonライブラリです。

## AUTOCREATE での活用
- OCR解析インターフェース
- AI視覚自動化デモ
- WIKI RAG システム
- セレクター分析ツール

## 実装例
```python
import gradio as gr

def ocr_analyze(image):
    # OCR処理
    return "OCR結果"

interface = gr.Interface(
    fn=ocr_analyze,
    inputs="image",
    outputs="text"
)
interface.launch()
```

## 利点
- 迅速なプロトタイピング
- 非技術者でも利用可能
- Colab連携
- 共有・デプロイが簡単
"""
        },
        {
            "filename": "ChromaDB-RAG.md",
            "content": """# ChromaDB RAGシステム

## 概要
ChromaDBは高性能なベクトルデータベースで、RAG（Retrieval-Augmented Generation）システムの構築に最適です。

## 特徴
- 高速ベクトル検索
- 永続化サポート
- 多様な埋め込みモデル対応
- シンプルなAPI

## AUTOCREATE での実装
1. **ナレッジベース構築**: 既存WIKIの自動ベクトル化
2. **質問応答システム**: 自然言語による検索・回答
3. **多言語対応**: 日本語・英語の同時対応

## 技術詳細
- **埋め込みモデル**: intfloat/multilingual-e5-large
- **チャンク分割**: RecursiveCharacterTextSplitter
- **検索手法**: コサイン類似度ベース
"""
        }
    ]
    
    # ファイル作成
    created_files = []
    for item in wiki_contents:
        file_path = os.path.join(sample_dir, item["filename"])
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(item["content"])
        created_files.append(file_path)
    
    print(f"📝 {len(created_files)}個のサンプルWIKIファイルを作成しました:")
    for file_path in created_files:
        print(f"  - {file_path}")
    
    return sample_dir

# サンプルデータ作成実行
sample_wiki_dir = create_sample_wiki_data()
print(f"\n✅ サンプルWIKIデータの準備が完了しました！")
print(f"📁 ディレクトリ: {sample_wiki_dir}")

In [ ]:
# 5️⃣ RAGシステムの初期化と構築
def build_rag_system():
    """RAGシステムの構築"""
    
    # サンプルWIKIパスを使用
    wiki_paths = [sample_wiki_dir]
    
    # RAGシステム初期化
    rag_system = WikiRAGSystem(wiki_paths=wiki_paths)
    
    # ChromaDB初期化
    print("🔧 ChromaDBを初期化中...")
    if not rag_system.initialize_chroma():
        print("❌ ChromaDB初期化に失敗しました")
        return None
    
    # WIKIドキュメント読み込み
    print("📚 WIKIドキュメントを読み込み中...")
    documents = []
    
    for wiki_path in wiki_paths:
        if os.path.exists(wiki_path):
            for filename in os.listdir(wiki_path):
                if filename.endswith('.md'):
                    file_path = os.path.join(wiki_path, filename)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                    
                    doc = Document(
                        page_content=content,
                        metadata={
                            'source': filename,
                            'source_type': 'wiki',
                            'wiki_path': wiki_path,
                            'loaded_at': datetime.now().isoformat()
                        }
                    )
                    documents.append(doc)
    
    print(f"📄 {len(documents)}個のドキュメントを読み込みました")
    
    # テキスト分割・処理
    print("✂️ テキストを分割中...")
    processed_docs = []
    
    for doc in documents:
        # Markdownを HTMLに変換してからテキスト抽出
        html = markdown.markdown(doc.page_content)
        soup = BeautifulSoup(html, 'html.parser')
        clean_text = soup.get_text()
        
        # テキスト分割
        chunks = rag_system.text_splitter.split_text(clean_text)
        
        for i, chunk in enumerate(chunks):
            if len(chunk.strip()) > 50:  # 短すぎるチャンクは除外
                chunk_doc = Document(
                    page_content=chunk,
                    metadata={
                        **doc.metadata,
                        'chunk_index': i,
                        'chunk_count': len(chunks)
                    }
                )
                processed_docs.append(chunk_doc)
    
    print(f"📝 {len(processed_docs)}個のチャンクに分割しました")
    
    # ベクトル化・格納
    print("🔄 ベクトル化・格納中... (これには少し時間がかかります)")
    rag_system.vectorstore.add_documents(processed_docs)
    
    # 統計情報表示
    final_count = rag_system.vectorstore._collection.count()
    print(f"✅ ナレッジベース構築完了！")
    print(f"📊 総ドキュメント数: {final_count}")
    
    return rag_system

# RAGシステム構築実行
print("🚀 RAGシステムを構築中...")
rag_system = build_rag_system()

if rag_system:
    print("\n🎉 RAGシステムの構築が完了しました！")
else:
    print("\n❌ RAGシステムの構築に失敗しました")

In [ ]:
# 6️⃣ 質問応答機能の実装
def search_knowledge(rag_system, query: str, k: int = 3):
    """ナレッジ検索"""
    if not rag_system or not rag_system.vectorstore:
        return []
    
    try:
        # 類似度検索
        results = rag_system.vectorstore.similarity_search_with_score(query, k=k)
        
        formatted_results = []
        for doc, score in results:
            formatted_results.append({
                'content': doc.page_content,
                'metadata': doc.metadata,
                'similarity_score': score,
                'source': doc.metadata.get('source', 'unknown')
            })
        
        return formatted_results
        
    except Exception as e:
        print(f"❌ 検索エラー: {e}")
        return []

def generate_answer(rag_system, question: str):
    """質問に対する回答生成"""
    print(f"🔍 質問: {question}")
    print("-" * 50)
    
    # 関連ナレッジ検索
    search_results = search_knowledge(rag_system, question, k=3)
    
    if not search_results:
        print("❌ 関連する情報が見つかりませんでした")
        return
    
    print("📚 関連ナレッジ:")
    for i, result in enumerate(search_results, 1):
        print(f"\n{i}. **{result['source']}** (類似度: {result['similarity_score']:.3f})")
        print(f"   {result['content'][:200]}...")
    
    # 最も関連性の高い情報で回答作成
    best_result = search_results[0]
    
    print(f"\n📝 **回答** (信頼度: {best_result['similarity_score']:.3f}):")
    print(best_result['content'])
    
    return {
        'answer': best_result['content'],
        'sources': search_results,
        'confidence': best_result['similarity_score']
    }

print("✅ 質問応答機能が実装されました！")

In [ ]:
# 7️⃣ 実際の質問デモ

# サンプル質問リスト
sample_questions = [
    "AUTOCREATEプロジェクトの特徴は何ですか？",
    "AI視覚自動化システムの技術スタックを教えてください",
    "Gradioの利点について説明してください",
    "ChromaDBの特徴は何ですか？",
    "OCRとRPAの組み合わせでできることは？"
]

print("🎯 サンプル質問による動作確認\n" + "="*60)

for i, question in enumerate(sample_questions, 1):
    print(f"\n【質問 {i}】")
    result = generate_answer(rag_system, question)
    print("\n" + "-"*60)

In [ ]:
# 8️⃣ インタラクティブ質問
def interactive_qa():
    """インタラクティブな質問応答"""
    print("💬 インタラクティブ質問モード")
    print("   質問を入力してください (終了するには 'quit' を入力)")
    print("-" * 50)
    
    while True:
        question = input("\n🤔 質問: ").strip()
        
        if question.lower() in ['quit', 'exit', '終了', 'q']:
            print("👋 お疲れ様でした！")
            break
        
        if not question:
            continue
        
        try:
            generate_answer(rag_system, question)
        except KeyboardInterrupt:
            print("\n\n👋 中断されました")
            break
        except Exception as e:
            print(f"❌ エラー: {e}")

# インタラクティブモードの説明
print("🎮 次のセルでインタラクティブな質問応答を試すことができます")
print("   以下のような質問を試してみてください:")
print("   - 'AI社長とは何ですか？'")
print("   - 'セレクター分析について教えて'") 
print("   - 'kinkaimasu.jp自動化の効果は？'")
print("\n⚠️  注意: Colabでは入力機能が制限される場合があります")

In [ ]:
# 9️⃣ Gradio Webインターフェース
def create_gradio_interface():
    """Gradio Webインターフェースの作成"""
    
    def qa_interface(question: str, max_results: int = 3):
        """Gradio用の質問応答インターフェース"""
        if not question.strip():
            return "質問を入力してください。", ""
        
        try:
            # 検索実行
            search_results = search_knowledge(rag_system, question, k=max_results)
            
            if not search_results:
                return "関連する情報が見つかりませんでした。", ""
            
            # 回答構築
            best_result = search_results[0]
            answer = f"""**質問:** {question}

**回答:** ({best_result['similarity_score']:.3f} 信頼度)
{best_result['content']}

**ソース:** {best_result['source']}
"""
            
            # 関連ソース情報
            sources_info = "**関連ソース:**\n"
            for i, result in enumerate(search_results, 1):
                sources_info += f"{i}. {result['source']} (類似度: {result['similarity_score']:.3f})\n"
                sources_info += f"   {result['content'][:100]}...\n\n"
            
            return answer, sources_info
            
        except Exception as e:
            return f"エラーが発生しました: {e}", ""
    
    # Gradioインターフェース定義
    with gr.Blocks(title="AUTOCREATE WIKI RAG", theme=gr.themes.Soft()) as interface:
        gr.Markdown("""
        # 🤖 AUTOCREATE WIKI RAG System
        
        **AI社長×無職CTO体制**による革新的なナレッジ検索システム
        
        既存のWIKIドキュメントから、あなたの質問に関連する情報を自動的に検索し、回答を生成します。
        """)
        
        with gr.Row():
            with gr.Column(scale=2):
                question_input = gr.Textbox(
                    label="質問",
                    placeholder="例: Gradioの使い方を教えてください",
                    lines=2
                )
                
                max_results = gr.Slider(
                    minimum=1,
                    maximum=10,
                    value=3,
                    step=1,
                    label="最大検索結果数"
                )
                
                submit_btn = gr.Button("🔍 質問する", variant="primary")
                
            with gr.Column(scale=1):
                gr.Markdown("""
                **サンプル質問:**
                - AUTOCREATEの特徴は？
                - AI視覚自動化とは？
                - ChromaDBの利点は？
                - OCR+RPAの効果は？
                """)
        
        with gr.Row():
            with gr.Column():
                answer_output = gr.Markdown(label="回答")
                
            with gr.Column():
                sources_output = gr.Markdown(label="関連ソース")
        
        # イベントハンドラ
        submit_btn.click(
            qa_interface,
            inputs=[question_input, max_results],
            outputs=[answer_output, sources_output]
        )
        
        question_input.submit(
            qa_interface,
            inputs=[question_input, max_results],
            outputs=[answer_output, sources_output]
        )
    
    return interface

# Gradioインターフェース作成
print("🌐 Gradio Webインターフェースを作成中...")
gradio_interface = create_gradio_interface()

# インターフェース起動
print("🚀 Webインターフェースを起動します...")
gradio_interface.launch(
    share=True,  # 公開URL生成
    debug=True,  # デバッグモード
    show_error=True
)

## 🚀 高度な機能と次のステップ

### 💡 このデモで学んだこと

1. **ChromaDB ベクトル検索**
   - 多言語対応の埋め込みモデル使用
   - 効率的なテキスト分割・チャンク化
   - 類似度ベースの関連文書検索

2. **RAG システム構築**
   - 既存ドキュメントの自動ベクトル化
   - 自然言語による質問応答
   - リアルタイム検索・回答生成

3. **Gradio Web UI**
   - 直感的なWebインターフェース
   - 共有可能なデモ環境
   - インタラクティブな質問応答

### 🔧 さらなる拡張可能性

- **大規模言語モデル統合**: OpenAI API, Anthropic API
- **マルチモーダル対応**: 画像・音声・動画の検索
- **リアルタイム更新**: 新しい文書の自動追加
- **パフォーマンス最適化**: GPU加速、分散処理

### 🌟 AUTOCREATE の革新性

**AI社長×無職CTO体制**により、従来の開発プロセスを根本から変革：

- 💭 **自然言語開発**: 思考をそのままコードに
- 🤖 **AI自動化**: 人間の創造性を最大化
- 🔄 **継続的進化**: システムが自己改良
- 🌐 **オープンソース**: 知識の共有と発展

---

### 📞 お問い合わせ・協業

このRAGシステムを実際のビジネスに活用したい場合は、お気軽にご相談ください。

**AUTOCREATE チーム**
- 🤖 AI社長: 戦略・ビジョン・意思決定
- 👨‍💻 無職CTO: 技術実装・システム設計